# Modeling Process
## Projecting Food Insecurity Rates in 2020
### Khyatee Desai

In [260]:
import pandas as pd
import numpy as np
from itertools import combinations
import statistics as stats
import scipy.stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
import sklearn
from sklearn.feature_selection import RFECV
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_regression
from statsmodels.stats.outliers_influence import variance_inflation_factor
import pickle
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

### Import cleaned dataset

In [292]:
with open('../pickled/feature_engineered_data.pickle', "rb") as input_file:
    df = pickle.load(input_file) 
# remove features not needed for modeling
df.drop(['FIPS', 'coc_number','FIPS_state', 'FIPS_county','Low Threshold Type', 'High Threshold Type',
       'State', 'County', 'State/County','Weighted Annual Dollars'],axis=1, inplace=True)

# Baseline Model
### Predicting on 2014 Data

In [293]:
# Limit the year and drop null values
df_14 = df[df.Year.isin(['2014'])].dropna()


### Train/Test Split

In [294]:
y = df_14['FI Rate']
X = df_14.drop([ 'FI Rate','Year'],axis=1)

# split into train test sets
X_train, X_test, y_train, y_test = train_test_split(X, y)

### Model

In [295]:
# Instantiate Linear Regression model
model_1 = LinearRegression()
model_1.fit(X_train, y_train)
y_train_pred = model_1.predict(X_train)
y_test_pred = model_1.predict(X_test)

# R2 of training and test set
print('R2 Train:',model_1.score(X_train, y_train))
print('R2 Test:',model_1.score(X_test, y_test))

# RMSE of training and test set
print('RMSE Train:',np.sqrt(mean_squared_error(y_train, y_train_pred)))
print('RMSE Test:',np.sqrt(mean_squared_error(y_test, y_test_pred)))

R2 Train: 0.6841778432384406
R2 Test: 0.44348064380809804
RMSE Train: 0.020580236899016023
RMSE Test: 0.03705036361169371


### Visually compare predictions

In [296]:
test_set = pd.concat([X_test, y_test],axis=1).reset_index()
df_preds = pd.concat([test_set,pd.Series(y_test_pred)],axis=1).rename(columns={0:'Y Test Preds'})
df_preds[['FI Rate','Y Test Preds']].sample(10)

,FI Rate,Y Test Preds
9,0.072,0.060123
12,0.116,0.086114
7,0.087,0.099791
0,0.162,0.171379
31,0.164,0.154204
33,0.063,0.098073
6,0.220,0.242603
2,0.172,0.164796
25,0.196,0.178537
19,0.094,0.080822


### Inspect Feature Importance

In [297]:
sorted(list(zip(model_1.coef_.tolist(), X_test.columns)), reverse=True)[:10]

[(0.4174887052915988, 'Percent_Black^2'),
 (0.2625770188476204, 'Percent_native'),
 (0.256295665649392, 'Cost Per Meal'),
 (0.13336683062404744, 'Percent_female'),
 (0.1332944664294207, 'Percent_female^2'),
 (0.10689343884101841, 'Percent_white'),
 (0.08259992566204633, 'Percent_pacific'),
 (0.045386897772676564, 'Sheltered_rate'),
 (0.012718811700125903, 'Percent_native^2'),
 (0.0007435516436294234, 'Unemployment_rate^2')]

# Feature Selection

In [298]:
df.columns

Index(['Rent', 'Year', 'Houseless_rate', 'Sheltered_rate', 'Unsheltered_rate',
       'TOT_POP', 'TOT_MALE', 'TOT_FEMALE', 'TOT_WHITE', 'TOT_BLACK',
       'TOT_NATIVE', 'TOT_ASIAN', 'TOT_PACIFIC', 'FI Rate', 'Cost Per Meal',
       'Num_wholesale', 'Num_restaraunts', 'Num_grocery', 'Total_workforce',
       'Employed', 'Unemployed', 'Unemployment_rate', 'Percent_male',
       'Percent_female', 'Percent_white', 'Percent_Black', 'Percent_native',
       'Percent_asian', 'Percent_pacific', 'Percent_working',
       'Total_food_retail', 'Food_retail_per_person', 'Rent^2',
       'Houseless_rate^2', 'Sheltered_rate^2', 'Unsheltered_rate^2',
       'TOT_POP^2', 'Cost Per Meal^2', 'Num_wholesale^2', 'Num_restaraunts^2',
       'Num_grocery^2', 'Unemployment_rate^2', 'Percent_male^2',
       'Percent_female^2', 'Percent_white^2', 'Percent_Black^2',
       'Percent_native^2', 'Percent_asian^2', 'Percent_pacific^2',
       'Percent_working^2', 'Total_food_retail^2', 'Food_retail_per_person^2'],

## Multicollinearity 
### Inspect correlation between each feature pair

In [194]:
corr=df_14.iloc[:,1:].corr().abs().stack().reset_index().sort_values(0, ascending=False)
corr['pairs'] = list(zip(corr.level_0, corr.level_1))
corr.drop(columns=['level_1', 'level_0'], inplace = True)
corr.columns = ['correlation', 'pairs']
corr.drop_duplicates(inplace=True)
corr[(corr.correlation >0.95) & (corr.correlation <1.0)]

,correlation,pairs
22328,1.000000,"(log_Percent_female, Percent_male^2)"
8041,1.000000,"(Percent_male^2, log_Percent_female)"
22172,1.000000,"(log_Percent_male, Percent_female^2)"
8356,1.000000,"(Percent_female^2, log_Percent_male)"
793,0.999947,"(TOT_FEMALE, TOT_POP)"
...,...,...
11634,0.951710,"(Houseless_rate_X_Percent_female, Houseless_ra..."
7845,0.950336,"(Unemployment_rate^3, Unemployment_rate_X_Perc..."
16323,0.950336,"(Unemployment_rate_X_Percent_male, Unemploymen..."
7307,0.950220,"(Num_grocery^2, TOT_POP^3)"


### VIF Scores

In [195]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X_train.values, i) for i in range(X_train.shape[1])]
vif["features"] = X_train.columns
vif_features = vif[vif['VIF Factor'] <10]['features']
vif[vif['VIF Factor'] <10]

,VIF Factor,features
1,1.200622,Houseless_rate
2,1.151213,Sheltered_rate
3,1.228337,Unsheltered_rate
8,6.506567,TOT_BLACK
11,1.446111,TOT_PACIFIC
...,...,...
154,1.148876,Hi_thresh_other
155,1.148876,Lo_thresh_SNAP
156,1.543336,Lo_thresh_SNAP_other
158,1.543341,Hi_thresh_SNAP_other


In [196]:
# Observe model performance using only features with low VIF
model_2 = LinearRegression()
model_2.fit(X_train[vif_features], y_train)
print(model_2.score(X_train[vif_features],y_train))      
print(model_2.score(X_test[vif_features],y_test)) 

0.9732181135710488
-1.4045687792261732


## K-Best Selector

In [280]:
##### Testing which new features actually help using K-Best:

## logs, interactions, dummies, and polynomials
# 6 0.6367829435769585
# 67 0.6724308721192602
# 68 0.6717160388094252

## logs, interactions, dummies 
# 16 0.7093947909784664

## interactions, dummies
# 42 0.792299764536843

## only dummies
# 10 0.7028965054037627

## no new features
# 10 0.5810599662789913

## only cubed polynomials
# 24 0.7765248697200026
#
## only squared polynomials
# 19 0.8136274183604031
#
#
#
# notes:
# 
#
#

In [299]:
# determine optimal k with a loop 

for k in range(1, 50):
    selector = SelectKBest(f_regression, k=k)
    selector.fit(X_train, y_train)
    kbest_features = X_train.columns[selector.get_support()]
    model_3 = LinearRegression()
    model_3.fit(X_train[kbest_features], y_train)
    y_test_pred = model_3.predict(X_test[kbest_features])
    print(k, model_3.score(X_test[kbest_features],y_test))

1 0.4364024601805663
2 0.4847462777605742
3 0.4866224230408247
4 0.48548017906013674
5 0.569651028859846
6 0.5411742665618392
7 0.6668203682147517
8 0.6550962914922812
9 0.654949603388506
10 0.6347415390826272
11 0.6347415390826321
12 0.6347415390823916
13 0.6777783759292214
14 0.6502099155988934
15 0.72292573888833
16 0.7916975775692484
17 0.7878319307598227
18 0.7878319450585183
19 0.8136274183604031
20 0.7928131807124577
21 0.767469325968424
22 0.753924793815939
23 0.7509444428275723
24 0.7555788414693709
25 0.7645115950187072
26 0.7486866652892701
27 0.7350373994230412
28 0.7126994627113635
29 0.7104320136418439
30 0.718509578383335
31 0.6864875219461892
32 0.6864875170307894
33 0.6472107443994342
34 0.6130387365841294
35 0.5698685719409632
36 0.49596140735919836
37 0.495110757019657
38 0.5093051291170487
39 0.4493783202728173
40 -0.469400900653802
41 0.16664710531379656
42 0.27791598358015845
43 0.2637713927501496
44 0.2876738086829671
45 0.4528058016686073
46 -0.08481095616269152

In [291]:
# run model with best 88 features
selector = SelectKBest(f_regression, k=24)
selector.fit(X_train, y_train)
kbest_features = X_train.columns[selector.get_support()]

model_3 = LinearRegression()
model_3.fit(X_train[kbest_features], y_train)
y_train_pred = model_3.predict(X_train[kbest_features])
y_test_pred = model_3.predict(X_test[kbest_features])

# R2 of training and test set
print('R2 Train:',model_3.score(X_train[kbest_features], y_train))
print('R2 Test:',model_3.score(X_test[kbest_features], y_test))

# RMSE of training and test set
print('RMSE Train:',np.sqrt(mean_squared_error(y_train, y_train_pred)))
print('RMSE Test:',np.sqrt(mean_squared_error(y_test, y_test_pred)))
kbest_features

R2 Train: 0.7603503561334701
R2 Test: 0.7765248697200026
RMSE Train: 0.018519824642181614
RMSE Test: 0.020066037333178407


Index(['Rent', 'Houseless_rate', 'Sheltered_rate', 'Unsheltered_rate',
       'TOT_BLACK', 'TOT_NATIVE', 'Unemployed', 'Unemployment_rate',
       'Percent_male', 'Percent_female', 'Percent_white', 'Percent_Black',
       'Percent_native', 'Percent_working', 'Food_retail_per_person', 'Rent^3',
       'Num_wholesale^3', 'Unemployment_rate^3', 'Percent_male^3',
       'Percent_female^3', 'Percent_white^3', 'Percent_Black^3',
       'Percent_native^3', 'Percent_working^3'],
      dtype='object')

## Recursive Feature Elimination

In [272]:
ols = LinearRegression()
selector = RFECV(estimator=ols, step=1, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
selector.fit(X_train, y_train)
rfe_features = X_train.columns[selector.support_]

In [273]:
model_4 = LinearRegression()
model_4.fit(X_train[rfe_features], y_train)
y_train_pred = model_4.predict(X_train[rfe_features])
y_test_pred = model_4.predict(X_test[rfe_features])

# R2 of training and test set
print('R2 Train:',model_4.score(X_train[rfe_features], y_train))
print('R2 Test:',model_4.score(X_test[rfe_features], y_test))

# RMSE of training and test set
print('RMSE Train:',np.sqrt(mean_squared_error(y_train, y_train_pred)))
print('RMSE Test:',np.sqrt(mean_squared_error(y_test, y_test_pred)))

R2 Train: 0.768717018926482
R2 Test: 0.6543222806901482
RMSE Train: 0.019911894109678422
RMSE Test: 0.021716092787100856
